In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [ ]:
class LeNet5(nn.Module):
    def __init__(self, activation):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # CIFAR10是3通道图像
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.activation = activation

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载CIFAR10数据集
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=False, transform=transform
)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=False, transform=transform
)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


In [ ]:
# 训练并记录损失
def train_and_record_loss(
        net, trainloader, criterion, optimizer, epochs=20
):
    losses = []
    accuracys = []
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 199:
                losses.append(running_loss / 200)
                running_loss = 0.0
        with torch.no_grad():
            true_count = 0
            total_count = 0
            for data in testloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total_count += labels.size(0)
                true_count += (predicted == labels).sum().item()
            accuracy = true_count / total_count
            accuracys.append(accuracy)
            # print(f'\rEpoch: {epoch + 1}, Accuracy: {accuracy:.4f}', end='')
    # print()
    return losses, accuracys


In [ ]:
net = LeNet5(torch.relu)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

losses, accuracys = train_and_record_loss(net, trainloader, criterion, optimizer)
print(f'loss: {losses[-1]}')
print(f'accuracy: {accuracys[-1]}')

In [ ]:
# 初始化网络、损失函数和优化器
net_relu = LeNet5(torch.relu)
net_sigmoid = LeNet5(torch.sigmoid)
net_tanh = LeNet5(torch.tanh)

criterion = nn.CrossEntropyLoss()
optimizer_relu = optim.Adam(net_relu.parameters(), lr=0.001)
optimizer_sigmoid = optim.Adam(net_sigmoid.parameters(), lr=0.001)
optimizer_tanh = optim.Adam(net_tanh.parameters(), lr=0.001)

# 训练并记录损失
losses_relu, accuracys_relu = train_and_record_loss(net_relu, trainloader, criterion, optimizer_relu)
losses_sigmoid, accuracys_sigmoid = train_and_record_loss(net_sigmoid, trainloader, criterion, optimizer_sigmoid)
losses_tanh, accuracys_tanh = train_and_record_loss(net_tanh, trainloader, criterion, optimizer_tanh)


In [ ]:
# 可视化损失
plt.plot(losses_relu, label='ReLU')
plt.plot(losses_sigmoid, label='Sigmoid')
plt.plot(losses_tanh, label='Tanh')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 可视化准确率
plt.plot(accuracys_relu, label='ReLU')
plt.plot(accuracys_sigmoid, label='Sigmoid')
plt.plot(accuracys_tanh, label='Tanh')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()